# Gastebois Jacques - Ei2i / Polytech
## Benchmarking : Validation de la Distillation sur Nouvelles Architectures

Comme j’ai pu le constater lors de mes missions chez Funky Junk, la polyvalence est la cle de la reussite technique. Ce notebook a pour but de valider que nos images distilles ne sont pas "sur-apprises" pour un seul modele, mais qu'elles peuvent servir de base d'entrainement pour n'importe quel reseau.

C'est l'essence meme de mon ambition : **DEVENIR INGENIEUR POUR LES INGENIEURS**. Fournir des outils robustes et verifies.

---

### 1. chargement des outils d'evaluation

on reprend les utilitaires du papier original pour etre sur de pas dire de betises sur les chiffres. j'ai un peu fait le menage dans les imports mais ca reste un peu le bazar. :)

In [ ]:
import torch
import torch.nn as  nn
import numpy as  np
import os 
from utils import get_dataset, get_network, evaluate_synset

if torch.cuda.is_available():
    dev = 'cuda'
elif torch.backends.mps.is_available():
	dev =  'mps' # pour mon m2 fetiche
else:
    dev = 'cpu'
    
print(f"on evalue sur {dev} !!")

### 2. recuperation des images et du modele

on va charger les images qu'on a sauvees dans le notebook 1. 
faut faire gaffe a bien donner le bon chemin. j'ai tendance a me perdre dans mes dossiers a force de faire de l'electronique et de l'info en meme temps. :p

In [ ]:
class eval_args:
    dataset = 'CIFAR10'
    model = 'ConvNet' # on peut changer pour ResNet18 si on veut rigoler
    epoch_eval_train = 1000 # temps d'entrainement sur le synset
    lr_net = 0.01
    batch_train = 256
    device = dev
    pca = False 
    zca = True # important si on l'a mis a la distillation !!!
    
config = eval_args()

# chemin vers les resultats du notebook 1
data_path = './logged_files/CIFAR10/ton_run_ici'
img_syn = torch.load(os.path.join(data_path, 'images_best.pt'), map_location=dev)
lab_syn = torch.load(os.path.join(data_path, 'labels_best.pt'), map_location=dev)

print("donnees chargees pour le benchmark :)")

### 3. entrainement et mesure de l'accuracy

on lance l'entrainement d'un nouveau modele a partir de zero, mais seulement sur nos 10 images (pour ipc=1). 
ensuite on le teste sur les vraies donnees de cifar10 qu'il n'a jamais vues. c'est le moment de verite. :o

In [ ]:
print("chargement du vrai set de test pour la validation...")
_, _, _, _, _, _, _, _, test_loader, _, _, _ = get_dataset(config.dataset, './data', 256, '', args=config)

def run_benchmark(model_name):
    print(f"\\nevaluation sur {model_name} en cours...")
    
    # on cree le reseau tout neuf
    net = get_network(model_name, 3, 10, (32,32)).to(dev)
    
    # on utilise la fonction evaluate du repo pour pas se planter dans les calculs
    # c'est pascal antonini qui m'a appris l'importance de la precision dans les mesures :)
    _, acc_train, acc_test = evaluate_synset(0, net, img_syn, lab_syn, test_loader, config)
    
    print(f"resultat final pour {model_name} :") 
	print( f"   accuracy train (sur le synset) : {acc_train:.2%}")
    print(f"   accuracy test (sur cifar10)  : {acc_test:.2%}")
    
    return acc_test

# on teste sur le modele de base
run_benchmark('ConvNet')

# si t'as le temps essaye avec ca
# run_benchmark('AlexNet') # c'est un peu plus lourd mais ca se fait :p

### 4. conclusion du benchmark

si on depasse les 40%, on est dans les clous du papier original. mon record est a 46.3%.
ca montre bien que meme avec tres peu de moyens (comme au debut de Funky Junk dans l'entrepot d'Ansiere), on peut faire du travail de qualite pro. 

merci a thibault hilaire pour ses conseils sur la partie systeme et a mme malonga pour l'aide sur la redaction du rapport. :)

Jacques Gastebois - 2026